In [1]:
import os

In [2]:
%pwd

'/home/mahi99/Documents/text_summarize_project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/mahi99/Documents/text_summarize_project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [6]:
from textSummerizer.constants import *
from textSummerizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath= PARMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])



    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir= config.root_dir,
            data_path= config.data_path,
            model_path= config.model_path,
            tokenizer_path= config.tokenizer_path,
            metric_file_name= config.metric_file_name

        )

        return model_evaluation_config
        

In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_from_disk, load_metric
import torch
import pandas as pd
from tqdm import tqdm

/home/mahi99/Documents/text_summarize_project/txtSum/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-07-22 17:12:17,328: INFO: config: PyTorch version 2.3.1 available.:]


In [11]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config



    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i: i + batch_size]


    
    def calculate_metric_on_test_ds(self, dataset, metric, model, tokenizer,
                                    batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu",
                                    column_text='article',
                                    column_summary='highlights'):
        
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batches, target_batches in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):


            inputs = tokenizer(article_batches, max_length=1024, truncation=True, padding='max_length', return_tensors='pt')
            summaries = model.generate(input_ids= inputs["input_ids"].to(device),
                            attention_mask=inputs['attention_mask'].to(device),
                            length_penalty=0.8, num_beams=8, max_length=128)
            

            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                    clean_up_tokenization_spaces=True)
                    for s in summaries]

            decoded_summaries = [d.replace(""," ") for d in decoded_summaries]
            

        metric.add_batch(predictions=decoded_summaries, references=target_batches)


        score = metric.compute()
        return score
    

    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)

        # loading dataset

        dataset_samsum_pt = load_from_disk(self.config.data_path)


        rouge_names = ['rouge1', "rouge2", "rougeL","rougeLsum"]
        rouge_metric = load_metric('rouge')

        score = self.calculate_metric_on_test_ds(
            dataset_samsum_pt['test'], rouge_metric, model_pegasus, tokenizer, batch_size=2, column_text = 'dialogue', column_summary= 'summary')


        rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)

        df=pd.DataFrame(rouge_dict, index=[f'pegasus'])
        df.to_csv(self.config.metric_file_name, index=False)


In [12]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.evaluate()
except Exception as e:
    raise e

[2024-07-22 17:26:24,006: INFO: common: yaml file:/home/mahi99/Documents/text_summarize_project/config/config.yaml loaded successfully:]
[2024-07-22 17:26:24,012: INFO: common: yaml file:/home/mahi99/Documents/text_summarize_project/params.yaml loaded successfully:]
[2024-07-22 17:26:24,014: INFO: common: created directory at: artifacts:]
[2024-07-22 17:26:24,015: INFO: common: created directory at: artifacts/model_evaluation:]


100%|██████████| 410/410 [2:34:25<00:00, 22.60s/it]  

[2024-07-22 20:00:54,103: INFO: rouge_scorer: Using default tokenizer.:]
